## Deep Learning
### Assignment 3
Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = '../data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
* data as a flat matrix,
* labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---

### Problem 1
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

In [11]:
batch_size = 128
lamda = 0.001

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  regul = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss += lamda * regul

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.100868
Minibatch accuracy: 10.2%
Validation accuracy: 14.8%
Minibatch loss at step 500: 2.579900
Minibatch accuracy: 78.9%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.665436
Minibatch accuracy: 80.5%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 1.014317
Minibatch accuracy: 82.0%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 0.827471
Minibatch accuracy: 86.7%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 0.867191
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.771752
Minibatch accuracy: 81.2%
Validation accuracy: 82.0%
Test accuracy: 89.0%


---

Regularization in Neural Network

In [17]:
batch_size = 128
lamda = 0.001

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases_1 = tf.Variable(tf.zeros([1024]))
  weights_2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(relu, weights_2) + biases_2

  regul = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) +\
          tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
  loss += lamda * regul
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  tf_valid_l1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  tf_valid_rl = tf.nn.relu(tf_valid_l1)
  tf_valid_l2 = tf.matmul(tf_valid_rl, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(tf_valid_l2)
  tf_test_l1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  tf_test_rl = tf.nn.relu(tf_test_l1)
  tf_test_l2 = tf.matmul(tf_test_rl, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tf_test_l2)

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 585.571411
Minibatch accuracy: 7.8%
Validation accuracy: 26.8%
Minibatch loss at step 500: 196.756363
Minibatch accuracy: 80.5%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 115.328964
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 68.869110
Minibatch accuracy: 90.6%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 41.505466
Minibatch accuracy: 89.1%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 25.375492
Minibatch accuracy: 87.5%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 15.534200
Minibatch accuracy: 84.4%
Validation accuracy: 86.3%
Test accuracy: 92.7%


---

### Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [19]:
batch_size = 4
lamda = 0.001

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases_1 = tf.Variable(tf.zeros([1024]))
  weights_2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(relu, weights_2) + biases_2

  regul = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) +\
          tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
  loss += lamda * regul
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  tf_valid_l1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  tf_valid_rl = tf.nn.relu(tf_valid_l1)
  tf_valid_l2 = tf.matmul(tf_valid_rl, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(tf_valid_l2)
  tf_test_l1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  tf_test_rl = tf.nn.relu(tf_test_l1)
  tf_test_l2 = tf.matmul(tf_test_rl, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tf_test_l2)

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 698.494873
Minibatch accuracy: 0.0%
Validation accuracy: 15.8%
Minibatch loss at step 500: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 50.0%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Test accuracy: 10.0%


---

### Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

In [25]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases_1 = tf.Variable(tf.zeros([1024]))
  weights_2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu = tf.nn.relu(logits_1)
  drop = tf.nn.dropout(relu, keep_prob=tf.placeholder(tf.float32))
  logits_2 = tf.matmul(drop, weights_2) + biases_2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
  tf_valid_l1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
  tf_valid_rl = tf.nn.relu(tf_valid_l1)
  tf_valid_l2 = tf.matmul(tf_valid_rl, weights_2) + biases_2
  valid_prediction = tf.nn.softmax(tf_valid_l2)
  tf_test_l1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
  tf_test_rl = tf.nn.relu(tf_test_l1)
  tf_test_l2 = tf.matmul(tf_test_rl, weights_2) + biases_2
  test_prediction = tf.nn.softmax(tf_test_l2)

In [27]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 'keep_prob' : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={'keep_prob': 1.0}), test_labels))

Initialized


TypeError: Cannot interpret feed_dict key as Tensor: The name 'keep_prob' looks like an (invalid) Operation name, not a Tensor. Tensor names must be of the form "<op_name>:<output_index>".

---